<a href="https://colab.research.google.com/github/PraveenBabu777/-praveen/blob/main/object%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

url = "https://plus.unsplash.com/premium_photo-1731629278058-4c9854a67d46?q=80&w=2940&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
image = Image.open(requests.get(url, stream=True).raw)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

Detected couch with confidence 0.994 at location [0.03, 3.02, 2938.68, 1936.45]
Detected dog with confidence 0.961 at location [385.49, 250.76, 1875.71, 1697.93]


In [8]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image, ImageDraw, ImageFont
import requests

# Load image from URL
url = "https://plus.unsplash.com/premium_photo-1687653086440-7dc61b5ff5db?q=80&w=3087&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
image = Image.open(requests.get(url, stream=True).raw)

# Load DETR model and processor
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

# Preprocess the image
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# Post-process outputs to obtain detected objects
target_sizes = torch.tensor([image.size[::-1]])  # Size as (height, width)
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

# Draw rectangles around detected objects
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]  # Convert to a list of rounded floats
    label_text = f"{model.config.id2label[label.item()]}: {round(score.item(), 3)}"
    print(f"Detected {label_text} at location {box}")

    # Draw rectangle
    draw.rectangle(box, outline="red", width=3)

    # Draw label
    draw.text((box[0], box[1] - 10), label_text, fill="red", font=font)

# Display the image
image.save('output.jpg')

Detected person: 0.997 at location [761.48, 2120.81, 1267.73, 3559.22]
Detected person: 0.999 at location [1152.04, 2320.58, 1760.96, 3878.75]
Detected backpack: 0.991 at location [1738.11, 2283.75, 2087.27, 2885.1]
Detected backpack: 0.989 at location [1323.95, 2646.06, 1746.22, 3295.44]
Detected person: 0.999 at location [1582.5, 1986.5, 2232.32, 3622.71]
